In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random
import tqdm
import pyautogui
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
import time 
import datetime

In [2]:
browser = webdriver.Chrome()


In [10]:
df = pd.DataFrame(columns = ['Что ищем','Время сбора','Ссылка','name','adr','work','ost','otz','osob','inf'])

In [4]:
browser.get(r'https://yandex.ru/maps/')

In [15]:
Poisk = browser.find_element(By.CLASS_NAME,'input__control')
adress = pd.read_excel('Книга1 (1).xlsx',header=None)
adress = adress[2:].reset_index(drop=True)
#adress = list(adress['адрес'])
adress

,0,1,2,3,4,5
0,Субъект РФ,Район региона,Город,Населенный пункт,Улица,Дом
1,Владимирская область,NaN,Город Петушки,NaN,Улица Ленина,Дом 12
2,Пермский край,NaN,Город Пермь,NaN,Улица Мира,Дом 45
3,Республика Татарстан,NaN,Город Нурлат,NaN,Улица Советская,Дом 94
4,Республика Мордовия,Чамзинский район,NaN,Рабочий поселок Чамзинка,Улица Победы,Дом 2
...,...,...,...,...,...,...
1436,Пермский край,Нытвенский район,NaN,Рабочий поселок Уральский,Улица Лесная,Дом 24
1437,Пермский край,Юрлинский район,NaN,Село Юрла,Улица Октябрьская,Дом 57
1438,Республика Татарстан,NaN,Город Казань,NaN,Улица Мавлютова,Дом 9
1439,Владимирская область,NaN,Город Владимир,NaN,Улица Комиссарова,Дом 6


In [17]:
for l in list(adress):
    adress[l] = adress[l].astype(str)
    adress[l] = adress[l].str.replace('nan','')

In [27]:
adress[0][2] + ' ' + adress[1][2] + ' ' + adress[2][2] + ' ' + adress[3][2] + ' ' + adress[4][2] + ' ' + adress[5][2]

'Пермский край  Город Пермь  Улица Мира Дом 45'

In [6]:
Poisk = browser.find_element(By.CLASS_NAME,'input__control')

In [134]:
adress = adress[9000:10000]

In [179]:
adress = ['175000, обл Новгородская, р-н Батецкий,  , п Батецкий, ул Первомайская, 27']

In [28]:
def sbor(aa):
    temp = pd.DataFrame(columns = ['Что ищем','Время сбора','Ссылка','name','adr','work','ost','otz','osob','inf'])
    time.sleep(1)
    
    work = ''
    
    if browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].text == 'Добавить':
        work = browser.find_elements(By.CLASS_NAME,'business-working-status-view')[-1].text
    elif browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].text == 'График':
        browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].click()
        time.sleep(1)
        #browser.find_elements(By.CLASS_NAME,'business-working-status-view')[0].click()
        time.sleep(2)
        e = browser.find_elements(By.CLASS_NAME,'business-working-intervals-view__item')
        
        for ee in e:
            work = work + ' ' + ee.text
        time.sleep(1)
    
    name = browser.find_elements(By.CLASS_NAME,'card-title-view__title-link')
    name = name[0].text
    
    adr = browser.find_elements(By.CLASS_NAME,'business-contacts-view__address-link')
    adr = adr[0].text

    try:
        ost = browser.find_element(By.CLASS_NAME,'masstransit-stops-view__stop-name').text
        metrs = browser.find_element(By.CLASS_NAME,'masstransit-stops-view__stop-distance').text
        ost_info = ost + ' ' + metrs
    except:
        ost_info = None
        

    features = browser.find_elements(By.CLASS_NAME,'business-features-view__bool-text')

    vkl = browser.find_elements(By.CLASS_NAME,'tabs-select-view__title')
    
    try:
        vkl[2].click()
        time.sleep(5)
    except:
        temp = temp.append({'Что ищем':aa,'Время сбора':datetime.datetime.now(),'Ссылка':'','name':'','adr':'','work':'','ost':'','otz':'','osob':'','inf':''},ignore_index=True)
        return temp

    o = browser.find_elements(By.CLASS_NAME,'business-review-view__body-text')
    otz = []
    for i in range(len(o)):
        otz.append(o[i].text)
    
    time.sleep(2)
    vkl[4].click()
    time.sleep(5)

    o = browser.find_elements(By.CLASS_NAME,'business-features-view__bool-text')
    osob = []
    for i in range(len(o)):
        osob.append(o[i].text)

    time.sleep(3)

    o = browser.find_elements(By.CLASS_NAME,'business-features-view__valued-value')
    oper = []
    for i in range(len(o)):
        oper.append(o[i].text)
    temp = temp.append({'Что ищем':aa,'Время сбора':datetime.datetime.now(),'Ссылка':browser.current_url,'name':name,'adr':adr,'work':work,'ost':ost_info,'otz':otz,'osob':osob,'inf':oper},ignore_index=True)
    return temp

c = 0
for aa in range(2,len(adress)):
    if c<0:
        pass
    else:
        Poisk.send_keys(Keys.CONTROL + 'a')
        Poisk.send_keys(Keys.DELETE)
        time.sleep(1)


        Poisk.send_keys('сбер отделение ' + adress[0][aa] + ' ' + adress[1][aa] + ' ' + adress[2][aa] + ' ' + adress[3][aa] + ' ' + adress[4][aa] + ' ' + adress[5][aa])
        time.sleep(2.5)
        Poisk.send_keys(Keys.ENTER)
        time.sleep(5)
        h = ''
        elem = browser.find_elements(By.CLASS_NAME,'search-business-snippet-view')
        if len(elem)>0:
            if len(elem)<10:
                if len(browser.find_elements(By.CLASS_NAME,'card-section-header__title'))>0:
                    df = df.append(pd.Series(), ignore_index=True)
                    df['Что ищем'][len(df)-1] = adress[0][aa] + ' ' + adress[1][aa] + ' ' + adress[2][aa] + ' ' + adress[3][aa] + ' ' + adress[4][aa] + ' ' + adress[5][aa]
                else:
                    while h != elem[-1]:
                        h = elem[-1]
                        browser.execute_script("arguments[0].scrollIntoView(true);",elem[-1])
                        time.sleep(1)
                        elem = browser.find_elements(By.CLASS_NAME,'search-business-snippet-view')


                    for h in elem:
                        browser.execute_script("arguments[0].scrollIntoView(true);",h)
                        time.sleep(1)
                        h.click()
                        df = df.append(sbor(adress[0][aa] + ' ' + adress[1][aa] + ' ' + adress[2][aa] + ' ' + adress[3][aa] + ' ' + adress[4][aa] + ' ' + adress[5][aa]))
                        break
            else:
                df = df.append(pd.Series(), ignore_index=True)
                df['Что ищем'][len(df)-1] = 'Больше 10 ' + adress[0][aa] + ' ' + adress[1][aa] + ' ' + adress[2][aa] + ' ' + adress[3][aa] + ' ' + adress[4][aa] + ' ' + adress[5][aa]

        else:
            if len(browser.find_elements(By.CLASS_NAME,'tabs-select-view__title'))<1:
                print(1)
                df = df.append(pd.Series(), ignore_index=True)
                df['Что ищем'][len(df)-1] = adress[0][aa] + ' ' + adress[1][aa] + ' ' + adress[2][aa] + ' ' + adress[3][aa] + ' ' + adress[4][aa] + ' ' + adress[5][aa]
            else:
                if len(browser.find_elements(By.CLASS_NAME,'nothing-found-view__header'))>0:
                    print(12)
                    df = df.append(pd.Series(), ignore_index=True)
                    df['Что ищем'][len(df)-1] = adress[0][aa] + ' ' + adress[1][aa] + ' ' + adress[2][aa] + ' ' + adress[3][aa] + ' ' + adress[4][aa] + ' ' + adress[5][aa]
                else:
                    print(10)
                    df = df.append(sbor(adress[0][aa] + ' ' + adress[1][aa] + ' ' + adress[2][aa] + ' ' + adress[3][aa] + ' ' + adress[4][aa] + ' ' + adress[5][aa]))
        
    break
    c = c + 1       

In [173]:
df.to_excel('Yandex 920-1000.xlsx',index=False)

In [29]:
df = df.reset_index(drop=True)

In [30]:
df

,Что ищем,Время сбора,Ссылка,name,adr,work,ost,otz,osob,inf
0,"г. Грозный ул. Ш-А Митаева дом 12 каб. 11, отд...",2024-12-26 09:49:31.599882,https://yandex.ru/maps/org/sberbank/1302733702...,СберБанк,"ул. Шейха Али Митаева, 12, Грозный",Понедельник\n09:00 – 18:00 Вторник\n09:00 – 1...,Волтайр 90 м,[Вывесели график в котором указано что в суббо...,"[, , , Данные Банка России, Обслуживание VIP-к...","[1481/1950/3, 08.12.2024, дополнительный офис,..."
1,Владимирская область Город Петушки Улица Лен...,2024-12-26 09:58:47.384810,https://yandex.ru/maps/org/sberbank/1013577472...,СберБанк,"ул. Ленина, 12, Петушки",Понедельник\n08:30 – 17:30 Вторник\n08:30 – 1...,Рынок 142 м,"[Получить заказ в постамате невозможно,постоян...","[, , , Данные Банка России, Обслуживание физич...","[1481/261/78, 08.12.2024, дополнительный офис,..."
2,Пермский край Город Пермь Улица Мира Дом 45,2024-12-26 10:15:35.640812,https://yandex.ru/maps/org/sberbank/2457612051...,СберБанк,"ул. Мира, 45, Пермь",Понедельник\n09:00 – 19:00 Вторник\n09:00 – 1...,Дворец культуры имени Гагарина 219 м,[Очень удобное отделение. Большой холл с банко...,"[, , Обслуживание физических лиц, Обслуживание...","[для корпоративных клиентов, для предпринимате..."


In [31]:
df['inf'][2]

['для корпоративных клиентов, для предпринимателей, среднего и малого бизнеса, для частных лиц',
 'ипотека, потребительский кредит',
 'покупка-продажа монет из драгоценных металлов, ксерокопирование, печать документов',
 'ТВ-приставка SberBox, оформление подписки СберПрайм, оформление Сим-карты СберМобайл, выдача интернет заказов, оформление ЭЦП от ФНС России']

In [118]:
vkl = browser.find_elements(By.CLASS_NAME,'tabs-select-view__title')

In [64]:
'отделение Сбербанка ' + adress[4].replace('отделение Сбербанка','').replace('.',' ').replace(',',' ')

'отделение Сбербанка Юрлинский район  с Усть-Зула  ул Центральная  41  '

In [185]:
df

,Что ищем,Время сбора,Ссылка,name,adr,work,ost,otz,osob,inf
0,"175000, обл Новгородская, р-н Батецкий, , п Б...",2024-03-21 11:30:26.723340,https://yandex.ru/maps/org/sberbank/2634055884...,Сбербанк,"Первомайская ул., 27, посёлок Батецкий",Понедельник\nКруглосуточно Вторник\nКруглосут...,Станция Батецкая 600 м,[],"[, Внесение наличных, Бесконтактные технологии...","[свободный, 30.09.2023, RUB, RUB, выдача налич..."
0,"175000, обл Новгородская, р-н Батецкий, , п Б...",2024-03-21 11:30:48.838089,https://yandex.ru/maps/org/sberbank/1030479122...,СберБанк,"Первомайская ул., 27, посёлок Батецкий",Понедельник\n08:30 – 12:0013:00 – 16:15 Вторн...,Станция Батецкая 610 м,"[Вежливый компетентный персонал , нет очереди ...","[, , , Данные Банка России, Обслуживание физич...","[1481/960/33, 11.03.2024, дополнительный офис,..."


In [189]:
df = df.reset_index()

In [191]:
df['inf'][1]

['1481/960/33',
 '11.03.2024',
 'дополнительный офис',
 'брокер, дилер, депозитарий, инвестиционный советник',
 '1481',
 'вклады в рублях, вклады в валюте, вклады в драгоценных металлах, операции с безналичной валютой, операции с наличной валютой, операции с драгоценными металлами',
 'действующая',
 'Sberbank of Russia, Sberbank',
 'Публичное акционерное общество "Сбербанк России"',
 'ПАО Сбербанк',
 '7707083893',
 '1027700132195',
 'для корпоративных клиентов, для предпринимателей, среднего и малого бизнеса, для частных лиц',
 'потребительский кредит']